In [1]:
from source.broker.trading_client import create_ibkr_client
from source.broker.trading_client import create_ibkr_client

In [2]:
%pip install ibapi
%pip install pandas numpy
%pip install talib-binary
%pip install ib_insync
%pip install matplotlib
%pip install yfinance
%pip install lightweight-charts

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement talib-binary (from versions: none)
ERROR: No matching distribution found for talib-binary
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import warnings
from lightweight_charts import Chart
warnings.filterwarnings('ignore')

In [4]:
from lightweight_charts import Chart
import pandas as pd

# Load data
print("Loading data...")
df = pd.read_csv('GC_checkpoint_100.csv')
df['date'] = pd.to_datetime(df['date'])
print(f"Loaded {len(df):,} bars")

# Pre-calculate all timeframes (more reliable)
print("Pre-calculating timeframes...")
timeframes = {}

# 1-minute (original)
timeframes['1min'] = pd.DataFrame({
    'time': df['date'],
    'open': df['open'],
    'high': df['high'],
    'low': df['low'],
    'close': df['close'],
    'volume': df['volume']
})

# 5-minute
df_5min = df.set_index('date').resample('5T').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum'
}).dropna()
timeframes['5min'] = pd.DataFrame({
    'time': df_5min.index,
    'open': df_5min['open'],
    'high': df_5min['high'],
    'low': df_5min['low'],
    'close': df_5min['close'],
    'volume': df_5min['volume']
})

# 15-minute
df_15min = df.set_index('date').resample('15T').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum'
}).dropna()
timeframes['15min'] = pd.DataFrame({
    'time': df_15min.index,
    'open': df_15min['open'],
    'high': df_15min['high'],
    'low': df_15min['low'],
    'close': df_15min['close'],
    'volume': df_15min['volume']
})

# 30-minute
df_30min = df.set_index('date').resample('30T').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum'
}).dropna()
timeframes['30min'] = pd.DataFrame({
    'time': df_30min.index,
    'open': df_30min['open'],
    'high': df_30min['high'],
    'low': df_30min['low'],
    'close': df_30min['close'],
    'volume': df_30min['volume']
})

# 1-hour
df_1h = df.set_index('date').resample('1H').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum'
}).dropna()
timeframes['1H'] = pd.DataFrame({
    'time': df_1h.index,
    'open': df_1h['open'],
    'high': df_1h['high'],
    'low': df_1h['low'],
    'close': df_1h['close'],
    'volume': df_1h['volume']
})

# 4-hour
df_4h = df.set_index('date').resample('4H').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum'
}).dropna()
timeframes['4H'] = pd.DataFrame({
    'time': df_4h.index,
    'open': df_4h['open'],
    'high': df_4h['high'],
    'low': df_4h['low'],
    'close': df_4h['close'],
    'volume': df_4h['volume']
})

# Daily
df_1d = df.set_index('date').resample('1D').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum'
}).dropna()
timeframes['1D'] = pd.DataFrame({
    'time': df_1d.index,
    'open': df_1d['open'],
    'high': df_1d['high'],
    'low': df_1d['low'],
    'close': df_1d['close'],
    'volume': df_1d['volume']
})

print("Timeframes ready:")
for tf, data in timeframes.items():
    print(f"  {tf}: {len(data):,} bars")


Loading data...
Loaded 93,592 bars
Pre-calculating timeframes...
Timeframes ready:
  1min: 93,592 bars
  5min: 18,719 bars
  15min: 6,240 bars
  30min: 3,120 bars
  1H: 1,561 bars
  4H: 425 bars
  1D: 87 bars


In [5]:
from lightweight_charts import Chart
import pandas as pd

# Load data once
df = pd.read_csv('GC_checkpoint_100.csv')
df['date'] = pd.to_datetime(df['date'])

print(f"Loaded {len(df):,} 1-minute bars")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")

def show_chart(timeframe='1min'):
    """
    Show chart with specific timeframe
    """
    print(f"\n📊 Loading {timeframe} chart...")
    
    # Resample based on timeframe
    if timeframe == '1min':
        chart_data = pd.DataFrame({
            'time': df['date'],
            'open': df['open'],
            'high': df['high'],
            'low': df['low'],
            'close': df['close'],
            'volume': df['volume']
        })
    else:
        # Timeframe mapping
        tf_map = {
            '5min': '5T',
            '15min': '15T',
            '30min': '30T',
            '1H': '1H',
            '4H': '4H',
            '1D': '1D'
        }
        
        df_resampled = df.set_index('date').resample(tf_map[timeframe]).agg({
            'open': 'first',
            'high': 'max',
            'low': 'min',
            'close': 'last',
            'volume': 'sum'
        }).dropna()
        
        chart_data = pd.DataFrame({
            'time': df_resampled.index,
            'open': df_resampled['open'],
            'high': df_resampled['high'],
            'low': df_resampled['low'],
            'close': df_resampled['close'],
            'volume': df_resampled['volume']
        })
    
    print(f"✓ {len(chart_data):,} bars")
    
    # Create chart
    chart = Chart()
    chart.set(chart_data)
    
    # Styling
    chart.legend(visible=True)
    chart.layout(background_color='#0c0d0f', text_color='#d1d4dc')
    chart.candle_style(
        up_color='#26a69a', down_color='#ef5350',
        border_up_color='#26a69a', border_down_color='#ef5350',
        wick_up_color='#26a69a', wick_down_color='#ef5350'
    )
    chart.volume_config(
        up_color='rgba(38, 166, 154, 0.5)',
        down_color='rgba(239, 83, 80, 0.5)'
    )
    
    chart.watermark(f'ES - {timeframe}')
    
    chart.show(block=True)

print("\n✓ Data loaded! Use these commands:")
print("  show_chart('1min')   - 1-minute bars")
print("  show_chart('5min')   - 5-minute bars")
print("  show_chart('15min')  - 15-minute bars")
print("  show_chart('30min')  - 30-minute bars")
print("  show_chart('1H')     - 1-hour bars")
print("  show_chart('4H')     - 4-hour bars")
print("  show_chart('1D')     - Daily bars")

Loaded 93,592 1-minute bars
Date range: 2025-08-13 18:00:00 to 2025-11-21 11:21:00

✓ Data loaded! Use these commands:
  show_chart('1min')   - 1-minute bars
  show_chart('5min')   - 5-minute bars
  show_chart('15min')  - 15-minute bars
  show_chart('30min')  - 30-minute bars
  show_chart('1H')     - 1-hour bars
  show_chart('4H')     - 4-hour bars
  show_chart('1D')     - Daily bars


In [8]:
# Chart is already open, just switch data:

show_chart('5min')   # Switches to 5-minute


📊 Loading 5min chart...
✓ 18,719 bars


AssertionError: cannot start a process twice